In [ ]:
import ROOT
import numpy as np

## Load AliRoot libraries

In [ ]:
ROOT.gSystem.Load("libSTAT")
ROOT.gSystem.Load("libCDB")
ROOT.gSystem.Load("libESD")

## Load OCDB

In [ ]:
ROOT.gROOT.LoadMacro("$ALICE_PHYSICS/PWGPP/TPC/macros/ConfigOCDB.C")  
ROOT.ConfigOCDB(129160,"local:///cvmfs/alice-ocdb.cern.ch/calibration/data/2010/OCDB/")

## Create example vector - default in python is list 
* we need C++ vectors

In [ ]:
nPoints=100
xyz=ROOT.TVectorD(3)
pxyz=ROOT.TVectorD(3)
cv=ROOT.TVectorD(15)
trdGeom = ROOT.AliTRDgeometry
layers=[]
for i in range(7):
    layers.append(trdGeom.GetTime0(i))
layers.append(400)  
layers.append(420)
pointsOut=ROOT.TMatrix(8,3)
pointsIn=ROOT.TMatrix(8,3)
massEl=0.0005
point=ROOT.TVectorD(2)
covPoint=ROOT.TVectorD(3)
covPoint[0]=0.05*0.05
covPoint[2]=1*1

In [ ]:
track0  = ROOT.AliExternalTrackParam(xyz.GetMatrixArray(),pxyz.GetMatrixArray(),cv.GetMatrixArray(),1)
layers

In [ ]:
def generateTrack():
    # generate intial track params and points along trajectory
    tanA=np.tan(np.pi/9)*np.random.uniform()
    pxyz[0]=np.random.uniform()
    pxyz[1]=pxyz[0]*tanA
    pxyz[2]=0;
    xyz[0]=trdGeom.GetTime0(0)
    xyz[1]=xyz[0]*tanA
    xyz[2]=0
    trackIn  = ROOT.AliExternalTrackParam(xyz.GetMatrixArray(),pxyz.GetMatrixArray(),cv.GetMatrixArray(),1)
    trackOut  = ROOT.AliExternalTrackParam(trackIn)   
    for layer in range(8):
        # Bool_t AliTrackerBase::PropagateTrackToBxByBz(AliExternalTrackParam* track, Double_t x, Double_t m, Double_t maxStep, Bool_t rotateTo = kTRUE, Double_t maxSnp = 0.80000000000000004, Int_t sign = 0, Bool_t addTimeStep = kFALSE, Bool_t correctMaterialBudget = kTRUE)
        ROOT.AliTrackerBase.PropagateTrackToBxByBz(trackOut,layers[i],  massEl, 1, 0, 0.8)
        pointsOut[layer,0] = trackOut.GetX()
        pointsOut[layer,1] = trackOut.GetY()
        pointsOut[layer,2] = trackOut.GetZ()
    trackFit  = ROOT.AliExternalTrackParam(trackOut)
    trackFits=[0,0,0,0,0,0,0,0,0,0]
    for layer in reversed(range(8)):
        # Bool_t AliTrackerBase::PropagateTrackToBxByBz(AliExternalTrackParam* track, Double_t x, Double_t m, Double_t maxStep, Bool_t rotateTo = kTRUE, Double_t maxSnp = 0.80000000000000004, Int_t sign = 0, Bool_t addTimeStep = kFALSE, Bool_t correctMaterialBudget = kTRUE)
        ROOT.AliTrackerBase.PropagateTrackToBxByBz(trackFit,layers[i],  massEl, 1, 0, 0.8)
        point[0]=pointsOut(layer,1)+np.random.normal()*0.05
        point[1]=pointsOut(layer,2)+np.random.normal()*1
        trackFit.Update(point.GetMatrixArray(),covPoint.GetMatrixArray())
        pointsIn[layer,0] = trackOut.GetX()
        pointsIn[layer,1] = trackOut.GetY()
        pointsIn[layer,2] = trackOut.GetZ()
        trackFits[layer]=ROOT.AliExternalTrackParam(trackFit)
    return trackIn, trackFits, points

In [ ]:
#trackIn, trackFit,dummy=generateTrack()

In [ ]:
#d = {'pt': [], 'sigmaPt0': [], 'sigmaPt0'}
for i in range(100):
    trackIn, trackFits,dummy=generateTrack()

In [ ]:
trdGeom.GetTime0(5)

In [ ]:
point[0]

In [ ]:
points.Print()

In [ ]:
# point[0]=pointsOut[0,1] #+ np.random.normal()*0.05
point[0]=pointsOut(0,1)